# ToDos:
    1. Remove cardinality (combine low occuring cat variables in columns into "Other")

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xgboost as xgb
import sklearn
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras import Sequential, layers, Model

In [15]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/train.csv")

train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
sample_submission = pd.read_csv("data/sample_submission.csv")
sample_submission

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


# Data Preprocessing
    - EDA is done in another notebook (EDA.ipynb)

In [133]:
# Create a copy of the training df to test preprocessing
data = train_df.copy()

In [134]:
def preprocess_data(df):
    # Drop columns with bad or unuseful data
    columns_to_drop = ["Id", "Street", "Alley","Utilities", "Condition1", "Condition2", "RoofMatl", "PoolQC", "MiscFeature"]
    df = df.drop(columns=columns_to_drop)
    
    # Transform some numerical columns to remove high skewness
    cols_to_transform = ["MasVnrArea", "BsmtFinSF1", "BsmtUnfSF", "2ndFlrSF", "WoodDeckSF",
                        "OpenPorchSF", "BsmtUnfSF", "BsmtFinSF2", "1stFlrSF", "2ndFlrSF", 
                        "GrLivArea", "MiscVal"]
    
    for i in cols_to_transform:
        df[i] = np.sqrt(df[i] + 1e-8)  
        
    # Reduce cardinality
    rarely_occuring = ["WdShing", "Stucco", "BrkComm", "AsbShng", "Stone", "ImStucc", "CBlock"]

    for i in rarely_occuring:
        df["Exterior1st"] = df["Exterior1st"].str.replace(i, "Other")
    
    return df
    
    
    

In [135]:
data = preprocess_data(data)
data

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,BldgType,...,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Reg,Lvl,Inside,Gtl,CollgCr,1Fam,...,0,0,0,NaN,0.0001,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Reg,Lvl,FR2,Gtl,Veenker,1Fam,...,0,0,0,NaN,0.0001,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,IR1,Lvl,Inside,Gtl,CollgCr,1Fam,...,0,0,0,NaN,0.0001,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,IR1,Lvl,Corner,Gtl,Crawfor,1Fam,...,0,0,0,NaN,0.0001,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,IR1,Lvl,FR2,Gtl,NoRidge,1Fam,...,0,0,0,NaN,0.0001,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Reg,Lvl,Inside,Gtl,Gilbert,1Fam,...,0,0,0,NaN,0.0001,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Reg,Lvl,Inside,Gtl,NWAmes,1Fam,...,0,0,0,MnPrv,0.0001,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Reg,Lvl,Inside,Gtl,Crawfor,1Fam,...,0,0,0,GdPrv,50.0000,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Reg,Lvl,Inside,Gtl,NAmes,1Fam,...,0,0,0,NaN,0.0001,4,2010,WD,Normal,142125


In [144]:
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore", sparse=False))
])

In [143]:
cat_data = data.select_dtypes(include="object")
cat_data

,MSZoning,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,BldgType,HouseStyle,RoofStyle,Exterior1st,...,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition
0,RL,Reg,Lvl,Inside,Gtl,CollgCr,1Fam,2Story,Gable,VinylSd,...,Typ,NaN,Attchd,RFn,TA,TA,Y,NaN,WD,Normal
1,RL,Reg,Lvl,FR2,Gtl,Veenker,1Fam,1Story,Gable,MetalSd,...,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,WD,Normal
2,RL,IR1,Lvl,Inside,Gtl,CollgCr,1Fam,2Story,Gable,VinylSd,...,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,WD,Normal
3,RL,IR1,Lvl,Corner,Gtl,Crawfor,1Fam,2Story,Gable,Wd Sdng,...,Typ,Gd,Detchd,Unf,TA,TA,Y,NaN,WD,Abnorml
4,RL,IR1,Lvl,FR2,Gtl,NoRidge,1Fam,2Story,Gable,VinylSd,...,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Reg,Lvl,Inside,Gtl,Gilbert,1Fam,2Story,Gable,VinylSd,...,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,WD,Normal
1456,RL,Reg,Lvl,Inside,Gtl,NWAmes,1Fam,1Story,Gable,Plywood,...,Min1,TA,Attchd,Unf,TA,TA,Y,MnPrv,WD,Normal
1457,RL,Reg,Lvl,Inside,Gtl,Crawfor,1Fam,2Story,Gable,CemntBd,...,Typ,Gd,Attchd,RFn,TA,TA,Y,GdPrv,WD,Normal
1458,RL,Reg,Lvl,Inside,Gtl,NAmes,1Fam,1Story,Hip,MetalSd,...,Typ,NaN,Attchd,Unf,TA,TA,Y,NaN,WD,Normal


In [138]:
pd.DataFrame(cat_pipeline.fit_transform(cat_data))

,0,1,2,3,4,5,6,7,8,9,...,198,199,200,201,202,203,204,205,206,207
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1456,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
